In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from myutils.utilities import *


In [94]:
pd.set_option('display.float_format', '{:.3f}'.format)


In [2]:
# txns_fn = '/home/claudio/tesi/sarafu/Sarafu2021_UKdb_submission/sarafu_xDAI/sarafu_txns_20200125-20210615.csv'
txns_fn = '/home/claudio/tesi/sarafu/final_data/txns.csv'
txns = pd.read_csv(txns_fn,sep=",",header=0,dtype={"id": "int",
                                                 "timeset": "string",
                                                 "transfer_subtype": "category",
                                                 'source': "string",
                                                 'target': "string",
                                                 'weight': "float",
                                                 'token_name': "category",
                                                 'token_address': "string",
                                                 })

txns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930161 entries, 0 to 930160
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   id                930161 non-null  int64   
 1   timeset           930161 non-null  string  
 2   transfer_subtype  930161 non-null  category
 3   source_xdai       930161 non-null  object  
 4   target_xdai       930161 non-null  object  
 5   weight            930161 non-null  float64 
 6   token_name        930161 non-null  category
 7   token_address     930161 non-null  string  
 8   date              930161 non-null  object  
 9   source            930161 non-null  string  
 10  target            930161 non-null  string  
dtypes: category(2), float64(1), int64(1), object(3), string(4)
memory usage: 65.6+ MB


In [3]:
# users_fn = '/home/claudio/tesi/sarafu/Sarafu2021_UKdb_submission/sarafu_xDAI/sarafu_users_20210615.csv'
users_fn = '/home/claudio/tesi/sarafu/final_data/users.csv'
users = pd.read_csv(users_fn,  # assuming 'user_fn' is your file name
                      sep=",",
                      header=0,
                      dtype={'id': 'int', 
                             'start': 'string', 
                             'final_bal': 'float', 
                             'gender': 'category', 
                             'area_name': 'string', 
                             'area_type': 'category',
                            'held_roles': 'category',
                            'business_type': 'category',
                            'old_POA_blockchain_address': 'string',
                            'xDAI_blockchain_address': 'string',
                            'ovol_in': 'float',
                            'ovol_out': 'float',
                            'otxns_in': 'int',
                            'otxns_out': 'int',
                            'ounique_in': 'int',
                            'ounique_out': 'int',
                            'svol_in': 'float',
                            'svol_out': 'float',
                            'stxns_in': 'int',
                            'stxns_out': 'int',
                            'sunique_in': 'int',
                            'sunique_out': 'int',
                            'crid': 'int'}) 

                            




In [5]:
users.loc[users.final_bal<0]

,id,start,final_bal,gender,area_name,area_type,held_roles,business_type,old_POA_blockchain_address,xDAI_blockchain_address,...,otxns_out,ounique_in,ounique_out,svol_in,svol_out,stxns_in,stxns_out,sunique_in,sunique_out,crid


In [51]:
PRECISION = 3 # number of decimal places to round

check if the data is clean

In [52]:
txns.columns[txns.isna().any()],users.columns[users.isna().any()]

(Index([], dtype='object'), Index(['final_bal'], dtype='object'))

rename columns

In [53]:
txns.drop(columns=['token_name','token_address','date'],inplace=True)
txns.rename(columns={"transfer_subtype": "type",'timeset': 'date'},inplace=True)
users.rename(columns={"held_roles": "role",
                          "business_type": "business",
                          "old_POA_blockchain_address": "poa",
                          "xDAI_blockchain_address": "xdai",},inplace=True)

# users.head()


Create 'Net cash flow' and 'Initial balance' column

In [54]:
all_out = users.svol_out + users.ovol_out
all_in = users.svol_in + users.ovol_in

net = all_in - all_out

users['net'] = net
# users['final_bal'] = users['final_bal'].fillna(0)
users['final_bal'] = users['final_bal'].fillna(abs(users['net']))

users['initial_bal'] = users['final_bal'] - net




In [55]:
users.columns[users.isna().any()]

Index([], dtype='object')

In [56]:
# Select columns with numeric types
numeric_cols = users.select_dtypes(include='number')

# Check which columns contain any values less than 0
columns_with_negative_values = numeric_cols.columns[(numeric_cols < 0).any(axis=0)]

print(columns_with_negative_values)


Index(['net', 'initial_bal'], dtype='object')


In [10]:
users[users.select_dtypes(include='float').columns] = users.select_dtypes(include='float').apply(
    lambda x: x.round(PRECISION)
)

users.initial_bal.value_counts()

NameError: name 'PRECISION' is not defined

In [57]:
mask = (users['final_bal'] < 0) | (users['initial_bal'] < 0)
users.loc[mask, ['final_bal', 'initial_bal']] = users.loc[mask, ['final_bal', 'initial_bal']].apply(
    lambda row: row - row.min(), axis=1
)

In [12]:
users[users.select_dtypes(include='float').columns] = users.select_dtypes(include='float').apply(
    lambda x: x.round(PRECISION)
)

users.initial_bal.value_counts()

NameError: name 'PRECISION' is not defined

In [58]:
txns['source_role'] = txns['source_xdai'].map(users.set_index('xdai')['role'])
txns['target_role'] = txns['target_xdai'].map(users.set_index('xdai')['role'])


txns['target_role'] = txns['target_role'].fillna('UnKnown')
txns['source_role'] = txns['source_role'].fillna('UnKnown')

first round of balance integration, backward 

In [59]:
tx_with_bal = txns.copy(deep=True)

In [60]:
previous_balance = {f'{address}': final_balance for address, final_balance in zip(users['crid'], users['final_bal'])}


In [ ]:
previous_balance = {f'{address}': final_balance for address, final_balance in zip(users['crid'], users['final_bal'])}

# Sort the DataFrame by date from newest to oldest
# tx_with_bal = all_tx.copy(deep=True)
tx_with_bal.sort_values(by='date', ascending=False, inplace=True)

# Iterate over the sorted DataFrame
source_bal_pre = []
target_bal_pre = []
source_bal_post = []
target_bal_post = []

for item in tx_with_bal.itertuples(index=True, name='Row'):
    # Set post-transaction balances
    source_bal_post.append(previous_balance[item.source])
    target_bal_post.append(previous_balance[item.target])

    # Calculate pre-transaction balances
    source_bal_pre.append(previous_balance[item.source] + item.weight)
    target_bal_pre.append(previous_balance[item.target] - item.weight)

    # Update the balances in the dictionary
    previous_balance[item.source] += item.weight
    previous_balance[item.target] -= item.weight

# Assign the computed lists back to the DataFrame
tx_with_bal['source_bal_pre'] = source_bal_pre
tx_with_bal['source_bal_post'] = source_bal_post
tx_with_bal['target_bal_pre'] = target_bal_pre
tx_with_bal['target_bal_post'] = target_bal_post




In [62]:
tx_with_bal=tx_with_bal[['id', 'date', 'type', 
                                 'source_xdai', 'target_xdai', 
                                 'weight',
                                 'source_bal_pre','source_bal_post', 
                                 'target_bal_pre', 'target_bal_post', 
                                 'source', 'target', 
                                 'source_role', 'target_role'
       ]]

In [63]:
tx_with_bal['right_weight'] = np.isclose(tx_with_bal['weight'], abs(tx_with_bal['source_bal_pre'] - tx_with_bal['source_bal_post']),atol=1e-3)
tx_with_bal.right_weight.value_counts()

right_weight
True    930161
Name: count, dtype: int64

In [64]:
tx_with_bal.sort_values(by='date', ascending=True, inplace=True)

In [21]:
THRESHOLD = 10**(-PRECISION)

float_cols = tx_with_bal.select_dtypes(include='float').columns

# Apply the lambda function to float columns
tx_with_bal[float_cols] = tx_with_bal[float_cols].apply(
    lambda x: x.apply(lambda y: 0 if abs(y) < THRESHOLD else y)
)
tx_with_bal

,id,date,type,source_xdai,target_xdai,weight,source_bal_pre,source_bal_post,target_bal_pre,target_bal_post,source,target,source_role,target_role,right_weight
0,1,2020-01-25 19:13:17.731529,DISBURSEMENT,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,0x245fc81fe385450Dc0f4787668e47c903C00b0A1,18000.000000,-10330585.919000,-10348585.919000,0.000000,18000.000000,8356,2,ADMIN,TOKEN_AGENT,True
1,2,2020-01-25 19:13:19.056070,DISBURSEMENT,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,0xC1697C1326fD192438515fE2F7E4cCb0C705C5d2,9047.661000,-10348585.919000,-10357633.580000,0.000000,9047.661000,8356,0,ADMIN,BENEFICIARY,True
2,3,2020-01-25 19:13:20.288346,DISBURSEMENT,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,0xBAB77A20a757e8438DfaBF01D5F36DD12d862B31,25378.726000,-10357633.580000,-10383012.306000,0.000000,25378.726000,8356,1,ADMIN,BENEFICIARY,True
3,4,2020-01-25 19:13:21.478850,DISBURSEMENT,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,0xD95954e3fCd2f09A6Be5931D24f731eFa63BF435,4495.933000,-10383012.306000,-10387508.239000,0.000000,4495.933000,8356,3,ADMIN,BENEFICIARY,True
4,5,2020-01-26 07:48:43.042684,DISBURSEMENT,0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F,0x4AB73CfaC1732a9DcD74BdB4C9605f21832D7C72,400.000000,43724689.390003,43724289.390003,0.000000,400.000000,8357,8358,ADMIN,BENEFICIARY,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930156,1035657,2021-06-15 04:46:20.990469,STANDARD,0x8581cd31b9a2FD9EB86fAA8c231c6E4167A47749,0x11C959A35453A793e05a19361782f3e13845708b,10.000000,95.000000,85.000000,200.800000,210.800000,45861,47568,BENEFICIARY,BENEFICIARY,True
930157,1035658,2021-06-15 04:47:58.072894,STANDARD,0x08637fE1D2D0F8FBdb2D3C2b8629aaD5E549A4b8,0x1CB6Ee7062C6Cd7CA8F0d71A84D7980b852E1648,10.000000,216.000000,206.000000,213.000000,223.000000,54370,44268,BENEFICIARY,BENEFICIARY,True
930158,1035659,2021-06-15 04:57:59.343271,STANDARD,0xafaFcc50bDB16eA7A6ecc69DEA267cc619E353d1,0xB88Cbf217F5b39762603aD7a6E0c86Bc0Bda1bb9,15.000000,254.000000,239.000000,246.200000,261.200000,43619,42955,BENEFICIARY,BENEFICIARY,True
930159,1035660,2021-06-15 04:59:54.897303,STANDARD,0xD6bcBcAfbE21F5Afe5B78EC71E5afBE2811608ad,0xC0bB86A110c331f569f9e81CB4f387cfBF12E4B4,10.000000,50.000000,40.000000,400.000000,410.000000,54891,51284,BENEFICIARY,BENEFICIARY,True


In [65]:
balances_pre = pd.DataFrame({
    'date': tx_with_bal['date'].repeat(2).values,  # Repeat each time twice
    'balance': tx_with_bal[['source_bal_pre', 'target_bal_pre']].values.flatten(),
    'xdai': tx_with_bal[['source_xdai', 'target_xdai']].values.flatten(),  # Stack feature1 and feature2 in an alternating pattern
    'type': tx_with_bal['type'].repeat(2).values
})

balances_post = pd.DataFrame({
    'date': tx_with_bal['date'].repeat(2).values,  # Repeat each time twice
    'balance': tx_with_bal[['source_bal_post', 'target_bal_post']].values.flatten(),
    'xdai': tx_with_bal[['source_xdai', 'target_xdai']].values.flatten(),  # Stack feature1 and feature2 in an alternating pattern
    'type': tx_with_bal['type'].repeat(2).values
})

In [23]:
balances_pre.balance = balances_pre.balance.apply(lambda x: 0 if abs(x) < THRESHOLD  else x)
balances_post.balance = balances_post.balance.apply(lambda x: 0 if abs(x) < THRESHOLD else x)
balances_post

,date,balance,xdai,type
0,2020-01-25 19:13:17.731529,-10348585.919000,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,DISBURSEMENT
1,2020-01-25 19:13:17.731529,18000.000000,0x245fc81fe385450Dc0f4787668e47c903C00b0A1,DISBURSEMENT
2,2020-01-25 19:13:19.056070,-10357633.580000,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,DISBURSEMENT
3,2020-01-25 19:13:19.056070,9047.661000,0xC1697C1326fD192438515fE2F7E4cCb0C705C5d2,DISBURSEMENT
4,2020-01-25 19:13:20.288346,-10383012.306000,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,DISBURSEMENT
...,...,...,...,...
1860317,2021-06-15 04:57:59.343271,261.200000,0xB88Cbf217F5b39762603aD7a6E0c86Bc0Bda1bb9,STANDARD
1860318,2021-06-15 04:59:54.897303,40.000000,0xD6bcBcAfbE21F5Afe5B78EC71E5afBE2811608ad,STANDARD
1860319,2021-06-15 04:59:54.897303,410.000000,0xC0bB86A110c331f569f9e81CB4f387cfBF12E4B4,STANDARD
1860320,2021-06-15 05:00:30.887568,96.960000,0x5aCA7747A43Eacfdca0A4C9D97826155dD4a432b,STANDARD


In [66]:
ser = balances_post.groupby('xdai').first('balance').value_counts()
ser 

balance   
50.000000     33496
400.000000     8315
400.000000      891
50.000000       797
50.000000       423
              ...  
159.946856        1
160.000000        1
160.027879        1
160.228874        1
158.871468        1
Name: count, Length: 2913, dtype: int64

In [67]:
balances_pre = balances_pre.merge(
    users[['xdai', 'role', 'area_type', 'area_name','svol_in', 'svol_out','stxns_in', 'stxns_out', 'sunique_in', 'sunique_out','initial_bal','final_bal','net','otxns_in', 'otxns_out','crid']],  # Ensure the right columns are selected
    on='xdai',  # Use 'xdai' as the merge key in users
    how='left'
)
balances_post = balances_post.merge(
    users[['xdai', 'role', 'area_type', 'area_name','svol_in', 'svol_out','stxns_in', 'stxns_out', 'sunique_in', 'sunique_out','initial_bal','final_bal','net','otxns_in', 'otxns_out','crid']],  # Ensure the right columns are selected
    on='xdai',  # Use 'xdai' as the merge key in users
    how='left'
)

FIRST round of validation 

On transactions sizes

In [68]:
tx_with_bal['right_weight'] = np.isclose(tx_with_bal['weight'], abs(tx_with_bal['source_bal_pre'] - tx_with_bal['source_bal_post']),atol=1e-3)
tx_with_bal.right_weight.value_counts()

right_weight
True    930161
Name: count, dtype: int64

On reconstructed vs calculated initial conditions

In [69]:
first = balances_pre.groupby('xdai').first()

first['validated'] = np.isclose(first.balance, first.initial_bal,atol = 1e-3)

first[first.validated == True]

# Optional: You can also count how many rows didn't validate
print(f'Total number of users: {len(first)}')
print(f"Number of validated balances: {len(first[first.validated == True])}")
print(f"Number of non-matching balances: {len(first[first.validated == False])}")

Total number of users: 54970
Number of validated balances: 54967
Number of non-matching balances: 3


In [70]:
first[first.validated == False]

,date,balance,type,role,area_type,area_name,svol_in,svol_out,stxns_in,stxns_out,sunique_in,sunique_out,initial_bal,final_bal,net,otxns_in,otxns_out,crid,validated
xdai,,,,,,,,,,,,,,,,,,,
0x9EE844A0cf2DD827604FC68Facb6c0259aF24CAe,2021-06-14 21:16:02.142453,-50.000000,DISBURSEMENT,BENEFICIARY,other,other,0.000000,20.000000,0,1,0,1,0.000000,30.000000,30.000000,1,0,54998,False
0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F,2020-01-26 07:48:43.042684,43724689.392073,DISBURSEMENT,ADMIN,other,other,0.000000,0.000000,0,0,0,0,27939762.420000,13969881.210000,-13969881.210000,0,219126,8357,False
0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,2020-01-25 19:13:17.731529,-10330585.934101,DISBURSEMENT,ADMIN,other,other,0.000000,0.000000,0,0,0,0,11730.000000,9000.000000,-2730.000000,0,8,8356,False


Looking for negative balances

In [71]:
neg_crid = list(set(tx_with_bal.loc[tx_with_bal.source_bal_pre < 0].source.unique()) | set(tx_with_bal.loc[tx_with_bal.source_bal_post < 0].source.unique()) | set(tx_with_bal.loc[tx_with_bal.target_bal_pre < 0].target.unique()) | set(tx_with_bal.loc[tx_with_bal.target_bal_post < 0].target.unique()))
negs_xdai = list(set(tx_with_bal.loc[tx_with_bal.source_bal_pre < 0].source_xdai.unique()) | set(tx_with_bal.loc[tx_with_bal.source_bal_post < 0].source_xdai.unique()) | set(tx_with_bal.loc[tx_with_bal.target_bal_pre < 0].target_xdai.unique()) | set(tx_with_bal.loc[tx_with_bal.target_bal_post < 0].target_xdai.unique()))


In [72]:
len(neg_crid)

2953

Second round of integration for users with negative balance

Assuming that accounts can't go below zero, we shift those balances by the minimum thay have ever reached 

In [73]:
grouped_pre = balances_pre.groupby('crid').balance.min()
grouped_pre = grouped_pre[grouped_pre.values<0]
grouped_post = balances_post.groupby('crid').balance.min()
grouped_post = grouped_post[grouped_post.values<0]
merged = pd.merge(grouped_pre,grouped_post,on='crid',how='outer')

merged['min_bal'] = merged[['balance_x','balance_y']].min(axis=1)

merged.reset_index(inplace=True)
merged

,crid,balance_x,balance_y,min_bal
0,3,-161.567424,-161.567424,-161.567424
1,6,-0.000000,NaN,-0.000000
2,7,-0.000000,NaN,-0.000000
3,8,-0.000000,NaN,-0.000000
4,24,-0.000000,NaN,-0.000000
...,...,...,...,...
2948,48874,-33.000000,-33.000000,-33.000000
2949,50038,-10.000000,-10.000000,-10.000000
2950,51306,-140.000000,-140.000000,-140.000000
2951,52530,-0.000000,NaN,-0.000000


In [74]:
users = users.merge(merged[['crid','min_bal']], on='crid', how='left')
users['min_bal'] = users['min_bal'].fillna(0)
users['final_bal'] = users['final_bal'] - users.min_bal
users['initial_bal'] = users['initial_bal'] - users.min_bal

In [75]:
tx_with_bal= txns.copy(deep=True)
previous_balance = {f'{address}': final_balance for address, final_balance in zip(users['crid'], users['final_bal'])}
# Sort the DataFrame by date from newest to oldest
# tx_with_bal= txns.copy(deep=True)
tx_with_bal.sort_values(by='date', ascending=False, inplace=True)

# Iterate over the sorted DataFrame
source_bal_pre = []
target_bal_pre = []
source_bal_post = []
target_bal_post = []

for item in tx_with_bal.itertuples(index=True, name='Row'):
    # Set post-transaction balances
    source_bal_post.append(previous_balance[item.source])
    target_bal_post.append(previous_balance[item.target])

    # Calculate pre-transaction balances
    source_bal_pre.append(previous_balance[item.source] + item.weight)
    target_bal_pre.append(previous_balance[item.target] - item.weight)

    # Update the balances in the dictionary
    previous_balance[item.source] += item.weight
    previous_balance[item.target] -= item.weight

# Assign the computed lists back to the DataFrame
tx_with_bal['source_bal_pre'] = source_bal_pre
tx_with_bal['source_bal_post'] = source_bal_post
tx_with_bal['target_bal_pre'] = target_bal_pre
tx_with_bal['target_bal_post'] = target_bal_post

tx_with_bal=tx_with_bal[['id', 'date', 'type', 
                                 'source_xdai', 'target_xdai', 
                                 'weight',
                                 'source_bal_pre','source_bal_post', 
                                 'target_bal_pre', 'target_bal_post', 
                                 'source', 'target', 
                                 'source_role', 'target_role'
       ]]
pd.set_option('display.float_format', '{:.6f}'.format)
tx_with_bal.sort_values(by='date', ascending=True, inplace=True)

threshold = 1e-3

# Apply the lambda function to all numeric columns
tx_with_bal[tx_with_bal.select_dtypes(include='number').columns] = tx_with_bal.select_dtypes(include='number').apply(
    lambda x: x.apply(lambda y: 0 if abs(y) < threshold else y)
)# tx_with_bal

In [76]:
tx_with_bal['right_weight'] = np.isclose(tx_with_bal['weight'], abs(tx_with_bal['source_bal_pre'] - tx_with_bal['source_bal_post']),atol=1e-3)
tx_with_bal.right_weight.value_counts()

right_weight
True    930161
Name: count, dtype: int64

In [77]:
balances_pre = pd.DataFrame({
    'date': tx_with_bal['date'].repeat(2).values,  # Repeat each time twice
    'balance': tx_with_bal[['source_bal_pre', 'target_bal_pre']].values.flatten(),
    'xdai': tx_with_bal[['source_xdai', 'target_xdai']].values.flatten(),  # Stack feature1 and feature2 in an alternating pattern
    'type': tx_with_bal['type'].repeat(2).values
})

balances_post = pd.DataFrame({
    'date': tx_with_bal['date'].repeat(2).values,  # Repeat each time twice
    'balance': tx_with_bal[['source_bal_post', 'target_bal_post']].values.flatten(),
    'xdai': tx_with_bal[['source_xdai', 'target_xdai']].values.flatten(),  # Stack feature1 and feature2 in an alternating pattern
    'type': tx_with_bal['type'].repeat(2).values
})

In [78]:
balances_pre.balance = balances_pre.balance.apply(lambda x: 0 if abs(x) < 1e-3 else x)
balances_post.balance = balances_post.balance.apply(lambda x: 0 if abs(x) < 1e-3 else x)
# balances_post

In [79]:
balances_post.groupby('xdai').first('balance').value_counts()


balance   
50.000000     34159
400.000000     8340
400.000000      901
50.000000       842
60.000000       409
              ...  
0.658059          1
0.778245          1
0.779390          1
0.806952          1
0.889802          1
Name: count, Length: 2879, dtype: int64

In [80]:
balances_pre = balances_pre.merge(
    users[['xdai', 'role', 'area_type', 'area_name','svol_in', 'svol_out','stxns_in', 'stxns_out', 'sunique_in', 'sunique_out','initial_bal','final_bal','net','otxns_in', 'otxns_out','crid']],  # Ensure the right columns are selected
    on='xdai',  # Use 'xdai' as the merge key in users
    how='left'
)
balances_post = balances_post.merge(
    users[['xdai', 'role', 'area_type', 'area_name','svol_in', 'svol_out','stxns_in', 'stxns_out', 'sunique_in', 'sunique_out','initial_bal','final_bal','net','otxns_in', 'otxns_out','crid']],  # Ensure the right columns are selected
    on='xdai',  # Use 'xdai' as the merge key in users
    how='left'
)

In [81]:
first = balances_pre.groupby('xdai').first()

first['validated'] = np.isclose(first.balance, first.initial_bal,atol = 1e-3)

first[first.validated == True]

# Optional: You can also count how many rows didn't validate
print(f'Total number of users: {len(first)}')
print(f"Number of validated balances: {len(first[first.validated == True])}")
print(f"Number of non-matching balances: {len(first[first.validated == False])}")

Total number of users: 54970
Number of validated balances: 54967
Number of non-matching balances: 3


In [82]:
not_validated = first[first.validated == False]

In [83]:
not_validated.crid

xdai
0x9EE844A0cf2DD827604FC68Facb6c0259aF24CAe    54998
0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F     8357
0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F     8356
Name: crid, dtype: int64

In [84]:
negs = list(set(tx_with_bal.loc[tx_with_bal.source_bal_pre < 0].source.unique()) | set(tx_with_bal.loc[tx_with_bal.source_bal_post < 0].source.unique()) | set(tx_with_bal.loc[tx_with_bal.target_bal_pre < 0].target.unique()) | set(tx_with_bal.loc[tx_with_bal.target_bal_post < 0].target.unique()))

negs

[]

Calculate spending and earning rates

In [85]:
tx_with_bal['frac_out'] = (tx_with_bal['source_bal_pre'] / tx_with_bal.weight)**(-1)
tx_with_bal['frac_in'] = (tx_with_bal['target_bal_pre'] / tx_with_bal.weight)**(-1)
tx_with_bal['frac_out_post'] = (tx_with_bal['source_bal_post'] / tx_with_bal.weight)**(-1)
tx_with_bal['frac_in_post'] = (tx_with_bal['target_bal_post'] / tx_with_bal.weight)**(-1)


In [86]:
import json 

dtype_mapping = {
    'int64': 'int',
    'float64': 'float',
    'object': 'string',
    'category': 'category'
}

In [87]:
dtype_dict = {col: dtype_mapping.get(str(tx_with_bal[col].dtype), str(tx_with_bal[col].dtype)) for col in tx_with_bal.columns}


dtype_dict

{'id': 'int',
 'date': 'string',
 'type': 'category',
 'source_xdai': 'string',
 'target_xdai': 'string',
 'weight': 'float',
 'source_bal_pre': 'float',
 'source_bal_post': 'float',
 'target_bal_pre': 'float',
 'target_bal_post': 'float',
 'source': 'string',
 'target': 'string',
 'source_role': 'category',
 'target_role': 'category',
 'right_weight': 'bool',
 'frac_out': 'float',
 'frac_in': 'float',
 'frac_out_post': 'float',
 'frac_in_post': 'float'}

In [88]:
dtype_dict = {col: dtype_mapping.get(str(balances_pre[col].dtype), str(balances_pre[col].dtype)) for col in balances_pre.columns}


dtype_dict

{'date': 'string',
 'balance': 'float',
 'xdai': 'string',
 'type': 'category',
 'role': 'category',
 'area_type': 'category',
 'area_name': 'string',
 'svol_in': 'float',
 'svol_out': 'float',
 'stxns_in': 'int',
 'stxns_out': 'int',
 'sunique_in': 'int',
 'sunique_out': 'int',
 'initial_bal': 'float',
 'final_bal': 'float',
 'net': 'float',
 'otxns_in': 'int',
 'otxns_out': 'int',
 'crid': 'int'}

In [89]:
dtype_dict = {col: dtype_mapping.get(str(users[col].dtype), str(users[col].dtype)) for col in users.columns}


dtype_dict

{'id': 'int',
 'start': 'string',
 'final_bal': 'float',
 'gender': 'category',
 'area_name': 'string',
 'area_type': 'category',
 'role': 'category',
 'business': 'category',
 'poa': 'string',
 'xdai': 'string',
 'ovol_in': 'float',
 'ovol_out': 'float',
 'otxns_in': 'int',
 'otxns_out': 'int',
 'ounique_in': 'int',
 'ounique_out': 'int',
 'svol_in': 'float',
 'svol_out': 'float',
 'stxns_in': 'int',
 'stxns_out': 'int',
 'sunique_in': 'int',
 'sunique_out': 'int',
 'crid': 'int',
 'net': 'float',
 'initial_bal': 'float',
 'min_bal': 'float'}

In [90]:
users[users.crid == 3]

,id,start,final_bal,gender,area_name,area_type,role,business,poa,xdai,...,svol_in,svol_out,stxns_in,stxns_out,sunique_in,sunique_out,crid,net,initial_bal,min_bal
3,5,2018-10-23 14:10:27,43389.850000,Male,Mukuru Nairobi,urban,BENEFICIARY,labour,0x3700dd6c99f396c48e6658102ab2e18dc776d243,0xD95954e3fCd2f09A6Be5931D24f731eFa63BF435,...,317816.300000,339096.000000,339,312,211,157,3,43228.282576,161.567424,-161.567424


In [91]:
tx_with_bal.to_csv('/home/claudio/tesi/sarafu/final_data/00_tx_with_bal_bwd_allpositivebalances.csv')
balances_post.to_csv('/home/claudio/tesi/sarafu/final_data/01_balances_post_bwd_allpositivebalances.csv')
balances_pre.to_csv('/home/claudio/tesi/sarafu/final_data/02_balances_pre_bwd_allpositivebalances.csv')
users.to_csv('/home/claudio/tesi/sarafu/final_data/03_users_adjusted_allpositivebalances.csv')

In [92]:
not_validated.to_csv()

'xdai,date,balance,type,role,area_type,area_name,svol_in,svol_out,stxns_in,stxns_out,sunique_in,sunique_out,initial_bal,final_bal,net,otxns_in,otxns_out,crid,validated\n0x9EE844A0cf2DD827604FC68Facb6c0259aF24CAe,2021-06-14 21:16:02.142453,0.0,DISBURSEMENT,BENEFICIARY,other,other,0.0,20.0,0,1,0,1,50.0,80.0,30.0,1,0,54998,False\n0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F,2020-01-26 07:48:43.042684,43724689.39207255,DISBURSEMENT,ADMIN,other,other,0.0,0.0,0,0,0,0,27939762.42,13969881.21,-13969881.21,0,219126,8357,False\n0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,2020-01-25 19:13:17.731529,4721876.586896978,DISBURSEMENT,ADMIN,other,other,0.0,0.0,0,0,0,0,15064192.520999946,15061462.520999946,-2730.0,0,8,8356,False\n'